# **Chapter 43: Build & Release**

---

## **Learning Objectives**

By the end of this chapter, you will be able to:

- Configure Android App Bundle (AAB) and APK signing for Google Play Store distribution
- Manage iOS certificates, provisioning profiles, and App Store Connect submissions
- Implement automated code signing and build processes using Fastlane
- Configure CI/CD pipelines with GitHub Actions and Codemagic for automated builds
- Execute integration and unit tests within CI/CD workflows
- Implement Over-the-Air (OTA) updates using modern alternatives to CodePush
- Manage versioning, build numbers, and release notes across platforms
- Configure beta testing via TestFlight and Google Play Internal Testing

---

## **Prerequisites**

- Completed Chapter 42: Security Best Practices (understanding of signing and certificates)
- Active Apple Developer Program membership (for iOS distribution)
- Google Play Developer account (for Android distribution)
- Understanding of YAML syntax for CI/CD configuration
- Familiarity with command-line tools and shell scripting
- Access to macOS hardware (for iOS builds, required by Apple)
- Understanding of semantic versioning (semver)

---

## **43.1 Build Configuration and Versioning**

Before releasing, configure build systems for production optimization and automated versioning.

### **Android Build Configuration**

```gradle
// android/app/build.gradle
android {
    namespace "com.example.battery_app"
    compileSdkVersion flutter.compileSdkVersion
    ndkVersion flutter.ndkVersion
    
    compileOptions {
        sourceCompatibility JavaVersion.VERSION_1_8
        targetCompatibility JavaVersion.VERSION_1_8
    }

    kotlinOptions {
        jvmTarget = '1.8'
    }

    defaultConfig {
        applicationId "com.example.battery_app"
        minSdkVersion 21
        targetSdkVersion 34
        
        // Version management - synced with pubspec.yaml
        versionCode flutterVersionCode.toInteger()
        versionName flutterVersionName
        
        // MultiDex for apps with many dependencies
        multiDexEnabled true
    }

    signingConfigs {
        release {
            // Load signing configuration from environment or properties
            // Never commit keystore credentials to version control
            storeFile file(System.getenv("ANDROID_KEYSTORE_PATH") ?: "release.keystore")
            storePassword System.getenv("ANDROID_KEYSTORE_PASSWORD")
            keyAlias System.getenv("ANDROID_KEY_ALIAS")
            keyPassword System.getenv("ANDROID_KEY_PASSWORD")
        }
        
        // Debug signing (auto-generated)
        debug {
            storeFile file('debug.keystore')
            storePassword 'android'
            keyAlias 'androiddebugkey'
            keyPassword 'android'
        }
    }

    buildTypes {
        release {
            signingConfig signingConfigs.release
            
            // Code shrinking and obfuscation
            minifyEnabled true
            shrinkResources true
            
            // ProGuard rules
            proguardFiles getDefaultProguardFile('proguard-android.txt'), 'proguard-rules.pro'
            
            // Enable R8 full mode for better optimization
            android.buildTypes.release.ndk.debugSymbolLevel = 'FULL'
        }
        
        debug {
            signingConfig signingConfigs.debug
            debuggable true
            applicationIdSuffix ".debug"
            versionNameSuffix "-debug"
        }
    }
    
    // App Bundle configuration for Play Store
    bundle {
        language {
            enableSplit = true
        }
        density {
            enableSplit = true
        }
        abi {
            enableSplit = true
        }
    }
}

flutter {
    source '../..'
}
```

**Explanation:**

- **versionCode**: Integer that must increment for each release. Google Play uses this to determine if an APK is an upgrade.
- **versionName**: Human-readable version string (e.g., "1.2.3") shown to users.
- **Signing Config**: Release builds must be signed with a private key. The keystore should never be committed to git; use environment variables or CI/CD secrets.
- **App Bundle (AAB)**: Modern Android format that defers APK generation to Google Play. Enables dynamic delivery (split APKs by language, density, ABI).
- **minifyEnabled**: Enables ProGuard/R8 code shrinking and obfuscation. Essential for security and reducing app size.
- **ndk.debugSymbolLevel**: Upload native debug symbols to Play Store for crash symbolication.

### **iOS Build Configuration**

```ruby
# ios/Podfile
platform :ios, '12.0'

# Disable CocoaPods stats for faster builds
ENV['COCOAPODS_DISABLE_STATS'] = 'true'

post_install do |installer|
  installer.pods_project.targets.each do |target|
    flutter_additional_ios_build_settings(target)
    
    target.build_configurations.each do |config|
      # Ensure consistent deployment target
      config.build_settings['IPHONEOS_DEPLOYMENT_TARGET'] = '12.0'
      
      # Code signing settings for CI/CD
      config.build_settings['CODE_SIGNING_REQUIRED'] = "NO"
      config.build_settings['CODE_SIGNING_ALLOWED'] = "NO"
      
      # Exclude simulator architectures for release builds
      if config.name == 'Release'
        config.build_settings['EXCLUDED_ARCHS[sdk=iphonesimulator*]'] = 'arm64'
      end
    end
  end
end
```

**Explanation:**

- **Deployment Target**: iOS 12.0 is the minimum recommended for Flutter 3.x. Lower versions may lack required APIs.
- **Code Signing**: Disabled in Podfile for CI/CD builds; Fastlane or Xcode will handle signing during export.
- **Architecture Exclusion**: Prevents building arm64 for simulators on Apple Silicon Macs when targeting physical devices.

---

## **43.2 Fastlane Automation**

Fastlane is the industry standard for automating mobile releases. It handles code signing, building, and uploading to stores.

### **Fastlane Setup**

```ruby
# Gemfile
source "https://rubygems.org"

gem "fastlane", "~> 2.217"
gem "cocoapods", "~> 1.14"
```

```bash
# Initialize Fastlane
cd android && fastlane init
cd ../ios && fastlane init
```

### **Android Fastlane Configuration**

```ruby
# android/fastlane/Fastfile
default_platform(:android)

platform :android do
  desc "Run unit tests"
  lane :test do
    gradle(task: "test")
  end

  desc "Build debug APK"
  lane :build_debug do
    gradle(
      task: "assemble",
      build_type: "Debug",
      properties: {
        "android.injected.version.code" => ENV["BUILD_NUMBER"],
        "android.injected.version.name" => ENV["VERSION_NAME"]
      }
    )
  end

  desc "Build release AAB"
  lane :build_release do
    # Validate keystore exists
    UI.user_error!("Keystore not found at #{ENV['ANDROID_KEYSTORE_PATH']}") unless File.exist?(ENV['ANDROID_KEYSTORE_PATH'])
    
    gradle(
      task: "bundle",
      build_type: "Release",
      properties: {
        "android.injected.signing.store.file" => ENV["ANDROID_KEYSTORE_PATH"],
        "android.injected.signing.store.password" => ENV["ANDROID_KEYSTORE_PASSWORD"],
        "android.injected.signing.key.alias" => ENV["ANDROID_KEY_ALIAS"],
        "android.injected.signing.key.password" => ENV["ANDROID_KEY_PASSWORD"],
        "android.injected.version.code" => ENV["BUILD_NUMBER"],
        "android.injected.version.name" => ENV["VERSION_NAME"]
      }
    )
    
    # Upload to Play Store
    upload_to_play_store(
      track: 'internal',
      aab: '../build/app/outputs/bundle/release/app-release.aab',
      release_status: 'draft',
      skip_upload_metadata: true,
      skip_upload_images: true,
      skip_upload_screenshots: true
    )
  end

  desc "Deploy to Play Store"
  lane :deploy do
    build_release
    
    upload_to_play_store(
      track: 'production',
      aab: '../build/app/outputs/bundle/release/app-release.aab',
      release_status: 'completed'
    )
  end
end
```

**Explanation:**

- **Gradle Properties**: Inject version codes and signing credentials via environment variables. Never hardcode in Fastfile.
- **AAB Generation**: `bundleRelease` generates Android App Bundle instead of APK. Required for new Play Store apps.
- **Play Store Tracks**: `internal` (fastest, up to 100 testers), `alpha`, `beta`, `production`.
- **Draft Status**: Upload as draft first to review before publishing.
- **Automatic Versioning**: Uses `BUILD_NUMBER` from CI/CD to increment versionCode automatically.

### **iOS Fastlane Configuration**

```ruby
# ios/fastlane/Fastfile
default_platform(:ios)

platform :ios do
  desc "Run tests"
  lane :test do
    scan(
      scheme: "Runner",
      device: "iPhone 15 Pro",
      clean: true
    )
  end

  desc "Sync certificates and provisioning profiles"
  lane :sync_signing do
    match(
      type: "appstore",
      readonly: true,
      app_identifier: "com.example.battery_app"
    )
  end

  desc "Build iOS release"
  lane :build do
    # Ensure CocoaPods dependencies are installed
    cocoapods(
      clean_install: true,
      use_bundle_exec: false
    )
    
    # Sync signing certificates
    match(
      type: "appstore",
      readonly: true,
      keychain_name: ENV["KEYCHAIN_NAME"],
      keychain_password: ENV["KEYCHAIN_PASSWORD"]
    )
    
    # Build archive
    gym(
      scheme: "Runner",
      export_method: "app-store",
      export_options: {
        provisioningProfiles: {
          "com.example.battery_app" => "match AppStore com.example.battery_app"
        }
      },
      build_path: "./build",
      output_directory: "./build/Runner",
      xcargs: "-allowProvisioningUpdates"
    )
  end

  desc "Upload to TestFlight"
  lane :beta do
    build
    
    upload_to_testflight(
      skip_waiting_for_build_processing: true,
      notify_external_testers: false,
      changelog: ENV["RELEASE_NOTES"] || "Bug fixes and improvements"
    )
  end

  desc "Deploy to App Store"
  lane :release do
    build
    
    upload_to_app_store(
      force: true,
      skip_metadata: false,
      skip_screenshots: false,
      submit_for_review: false,
      automatic_release: false,
      precheck_include_in_app_purchases: false
    )
  end
end
```

```ruby
# ios/fastlane/Matchfile
# Configuration for code signing via match
storage_mode("git")
type("appstore")
app_identifier(["com.example.battery_app"])
username("developer@example.com")
git_url("https://github.com/yourcompany/certificates")
git_branch("main")
```

**Explanation:**

- **Match**: Fastlane's encrypted git storage for certificates and provisioning profiles. Eliminates "code signing hell" by sharing credentials securely via private git repo.
- **Gym**: Builds and packages the iOS app. Generates IPA or uploads directly.
- **TestFlight**: Apple's beta testing platform. `skip_waiting_for_build_processing` returns immediately; processing takes 10-30 minutes.
- **App Store Upload**: Includes metadata, screenshots, and app review information.
- **Keychain**: CI/CD systems create temporary keychains for signing. Must be unlocked with password.

---

## **43.3 CI/CD Pipeline Configuration**

Automate builds, tests, and deployments using GitHub Actions or Codemagic.

### **GitHub Actions Workflow**

```yaml
# .github/workflows/build-release.yml
name: Build and Release

on:
  push:
    branches: [main]
    tags: ['v*']
  pull_request:
    branches: [main]

env:
  FLUTTER_VERSION: '3.16.0'
  RUBY_VERSION: '3.2'

jobs:
  test:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4
      
      - name: Setup Flutter
        uses: subosito/flutter-action@v2
        with:
          flutter-version: ${{ env.FLUTTER_VERSION }}
          channel: 'stable'
          cache: true
      
      - name: Get dependencies
        run: flutter pub get
      
      - name: Run analyzer
        run: flutter analyze
      
      - name: Run tests
        run: flutter test --coverage
      
      - name: Upload coverage
        uses: codecov/codecov-action@v3
        with:
          files: ./coverage/lcov.info

  build-android:
    needs: test
    runs-on: ubuntu-latest
    if: startsWith(github.ref, 'refs/tags/v')
    
    steps:
      - uses: actions/checkout@v4
      
      - name: Setup Flutter
        uses: subosito/flutter-action@v2
        with:
          flutter-version: ${{ env.FLUTTER_VERSION }}
          channel: 'stable'
      
      - name: Setup Ruby
        uses: ruby/setup-ruby@v1
        with:
          ruby-version: ${{ env.RUBY_VERSION }}
          bundler-cache: true
      
      - name: Setup Java
        uses: actions/setup-java@v3
        with:
          distribution: 'zulu'
          java-version: '17'
      
      - name: Decode keystore
        run: |
          echo "${{ secrets.ANDROID_KEYSTORE_BASE64 }}" | base64 --decode > android/release.keystore
      
      - name: Build AAB
        run: |
          cd android
          bundle install
          bundle exec fastlane build_release
        env:
          ANDROID_KEYSTORE_PATH: release.keystore
          ANDROID_KEYSTORE_PASSWORD: ${{ secrets.ANDROID_KEYSTORE_PASSWORD }}
          ANDROID_KEY_ALIAS: ${{ secrets.ANDROID_KEY_ALIAS }}
          ANDROID_KEY_PASSWORD: ${{ secrets.ANDROID_KEY_PASSWORD }}
          BUILD_NUMBER: ${{ github.run_number }}
          VERSION_NAME: ${{ github.ref_name }}
      
      - name: Upload AAB
        uses: actions/upload-artifact@v3
        with:
          name: android-release
          path: build/app/outputs/bundle/release/*.aab

  build-ios:
    needs: test
    runs-on: macos-13  # macOS required for iOS builds
    if: startsWith(github.ref, 'refs/tags/v')
    
    steps:
      - uses: actions/checkout@v4
      
      - name: Setup Flutter
        uses: subosito/flutter-action@v2
        with:
          flutter-version: ${{ env.FLUTTER_VERSION }}
          channel: 'stable'
      
      - name: Setup Ruby
        uses: ruby/setup-ruby@v1
        with:
          ruby-version: ${{ env.RUBY_VERSION }}
          bundler-cache: true
      
      - name: Install dependencies
        run: |
          cd ios
          bundle install
          pod install --repo-update
      
      - name: Setup keychain
        run: |
          security create-keychain -p "${{ secrets.KEYCHAIN_PASSWORD }}" build.keychain
          security default-keychain -s build.keychain
          security unlock-keychain -p "${{ secrets.KEYCHAIN_PASSWORD }}" build.keychain
          security set-keychain-settings -t 3600 -u build.keychain
      
      - name: Sync certificates
        run: |
          cd ios
          bundle exec fastlane sync_signing
        env:
          MATCH_PASSWORD: ${{ secrets.MATCH_PASSWORD }}
          MATCH_GIT_BASIC_AUTHORIZATION: ${{ secrets.MATCH_GIT_BASIC_AUTHORIZATION }}
      
      - name: Build iOS
        run: |
          cd ios
          bundle exec fastlane beta
        env:
          KEYCHAIN_NAME: build.keychain
          KEYCHAIN_PASSWORD: ${{ secrets.KEYCHAIN_PASSWORD }}
          RELEASE_NOTES: "Release ${{ github.ref_name }}"
      
      - name: Upload IPA
        uses: actions/upload-artifact@v3
        with:
          name: ios-release
          path: ios/build/Runner/*.ipa
```

**Explanation:**

- **Trigger Conditions**: Run tests on all PRs; build releases only on version tags (`v1.0.0`).
- **Flutter Action**: Caches Flutter SDK for faster builds.
- **Secrets Management**: Keystore and certificates stored as GitHub Secrets, injected as environment variables.
- **Base64 Encoding**: Binary files (keystore) must be base64 encoded for storage in secrets.
- **macOS Requirement**: iOS builds must run on macOS runners (GitHub provides macos-13, macos-14).
- **Keychain Setup**: macOS requires keychain creation and unlocking for code signing in CI.
- **Match Authorization**: GitHub token for accessing private certificates repository.

### **Codemagic Configuration**

```yaml
# codemagic.yaml
workflows:
  release-workflow:
    name: Release Build
    instance_type: mac_mini_m1  # M1 for faster iOS builds
    max_build_duration: 60
    
    environment:
      flutter: stable
      xcode: latest
      cocoapods: default
      java: 17
      
      groups:
        - google_play_credentials
        - app_store_credentials
        - match_credentials
    
    cache:
      cache_paths:
        - $FLUTTER_ROOT/.pub-cache
        - $HOME/.gradle/caches
        - ios/Pods
    
    triggering:
      events:
        - tag
      tag_patterns:
        - pattern: 'v*'
          include: true
    
    scripts:
      - name: Get Flutter packages
        script: flutter packages pub get
      
      - name: Run tests
        script: |
          flutter test
          flutter analyze
      
      - name: Setup Android signing
        script: |
          echo $ANDROID_KEYSTORE | base64 --decode > android/app/keystore.jks
          cat >> android/local.properties <<EOF
          keystore.path=keystore.jks
          keystore.password=$ANDROID_KEYSTORE_PASSWORD
          key.alias=$ANDROID_KEY_ALIAS
          key.password=$ANDROID_KEY_PASSWORD
          EOF
      
      - name: Build Android AAB
        script: |
          flutter build appbundle \
            --release \
            --build-number=$(date +%s) \
            --build-name=${CM_TAG#v}
      
      - name: Build iOS
        script: |
          cd ios
          pod install --repo-update
          flutter build ipa --release
      
      - name: Publish to Play Store
        script: |
          cd android
          bundle exec fastlane deploy
        ignore_failure: true  # Continue even if Play Store upload fails
      
      - name: Publish to App Store
        script: |
          cd ios
          bundle exec fastlane release
        ignore_failure: true

    artifacts:
      - build/app/outputs/bundle/release/*.aab
      - build/ios/ipa/*.ipa
      - ios/build/Runner/*.ipa
```

**Explanation:**

- **Instance Type**: `mac_mini_m1` builds iOS apps and is faster than Intel for Flutter builds.
- **Environment Groups**: Codemagic's secret management system. Groups contain related credentials (Play Store service account, App Store API key).
- **Tag Triggering**: Only builds on git tags matching `v*`.
- **Build Number**: Uses Unix timestamp for unique, incrementing build number.
- **Ignore Failure**: Allows iOS upload to continue even if Android fails (and vice versa).
- **Caching**: Caches Pub dependencies, Gradle, and CocoaPods for faster subsequent builds.

---

## **43.4 Over-the-Air (OTA) Updates**

Since Microsoft discontinued CodePush, modern alternatives like Shorebird or custom solutions are used for hotfixes without app store review.

### **Shorebird Integration**

```yaml
# pubspec.yaml
dependencies:
  flutter:
    sdk: flutter
  shorebird_code_push: ^1.1.0  # Optional: for checking update status
```

```dart
// lib/services/update_service.dart
import 'package:shorebird_code_push/shorebird_code_push.dart';

/// Service for managing over-the-air updates via Shorebird
class UpdateService {
  final ShorebirdCodePush _shorebird = ShorebirdCodePush();
  
  /// Check if current patch is the latest
  Future<bool> isUpToDate() async {
    try {
      return await _shorebird.isUpToDate();
    } catch (e) {
      print('Failed to check update status: $e');
      return true; // Assume up to date on error
    }
  }
  
  /// Get current patch number
  Future<int?> getCurrentPatch() async {
    return await _shorebird.currentPatchNumber();
  }
  
  /// Check for and install updates
  Future<UpdateResult> checkForUpdate() async {
    try {
      final isUpdateAvailable = await _shorebird.checkForUpdate();
      
      if (!isUpdateAvailable) {
        return UpdateResult.noUpdate;
      }
      
      // Download and install in background
      await _shorebird.downloadUpdateIfAvailable();
      
      return UpdateResult.updateInstalled;
    } catch (e) {
      return UpdateResult.error(e.toString());
    }
  }
  
  /// Prompt user to restart to apply update
  void showUpdatePrompt(BuildContext context) {
    showDialog(
      context: context,
      builder: (context) => AlertDialog(
        title: Text('Update Available'),
        content: Text('A new update has been downloaded. Restart to apply?'),
        actions: [
          TextButton(
            onPressed: () => Navigator.pop(context),
            child: Text('Later'),
          ),
          ElevatedButton(
            onPressed: () {
              // Restart app to apply patch
              _shorebird.restartApp();
            },
            child: Text('Restart Now'),
          ),
        ],
      ),
    );
  }
}

enum UpdateResult {
  noUpdate,
  updateInstalled,
  error,
}
```

**Explanation:**

- **Shorebird**: Flutter-specific OTA solution using code push technology. Patches Dart code (not native code) without app store review.
- **Patch Numbers**: Each OTA update increments patch number (e.g., 1.0.0+1, 1.0.0+2).
- **Download Strategy**: Updates download in background; app restarts to apply.
- **Limitations**: Cannot update native code (Android/iOS plugins), only Dart code. Major version changes still require store submission.
- **Security**: Shorebird uses code signing to ensure patches come from authorized sources.

### **Custom In-App Update Flow**

```dart
// lib/widgets/update_checker.dart
import 'package:flutter/material.dart';
import 'package:package_info_plus/package_info_plus.dart';
import 'package:http/http.dart' as http;
import 'dart:convert';

/// Checks for mandatory store updates (not OTA)
class StoreUpdateChecker {
  static const String appStoreId = '1234567890';
  static const String playStorePackage = 'com.example.battery_app';
  
  static Future<StoreVersion?> checkForStoreUpdate() async {
    final packageInfo = await PackageInfo.fromPlatform();
    final currentVersion = packageInfo.version;
    
    if (Platform.isAndroid) {
      return await _checkPlayStore(currentVersion);
    } else if (Platform.isIOS) {
      return await _checkAppStore(currentVersion);
    }
    return null;
  }
  
  static Future<StoreVersion?> _checkPlayStore(String currentVersion) async {
    try {
      // Note: Play Store doesn't have official API for version checking
      // Use custom backend or scraper (with caution)
      final response = await http.get(
        Uri.parse('https://your-api.com/version/android'),
      );
      
      if (response.statusCode == 200) {
        final data = json.decode(response.body);
        final latestVersion = data['version'] as String;
        final isMandatory = data['mandatory'] as bool? ?? false;
        
        if (_isNewer(latestVersion, currentVersion)) {
          return StoreVersion(
            version: latestVersion,
            isMandatory: isMandatory,
            storeUrl: 'market://details?id=$playStorePackage',
          );
        }
      }
    } catch (e) {
      print('Failed to check Play Store: $e');
    }
    return null;
  }
  
  static Future<StoreVersion?> _checkAppStore(String currentVersion) async {
    try {
      final response = await http.get(
        Uri.parse('https://itunes.apple.com/lookup?id=$appStoreId'),
      );
      
      if (response.statusCode == 200) {
        final data = json.decode(response.body);
        final results = data['results'] as List;
        
        if (results.isNotEmpty) {
          final latestVersion = results[0]['version'] as String;
          
          if (_isNewer(latestVersion, currentVersion)) {
            return StoreVersion(
              version: latestVersion,
              isMandatory: false,
              storeUrl: 'https://apps.apple.com/app/id$appStoreId',
            );
          }
        }
      }
    } catch (e) {
      print('Failed to check App Store: $e');
    }
    return null;
  }
  
  static bool _isNewer(String latest, String current) {
    final latestParts = latest.split('.').map(int.parse).toList();
    final currentParts = current.split('.').map(int.parse).toList();
    
    for (var i = 0; i < latestParts.length && i < currentParts.length; i++) {
      if (latestParts[i] > currentParts[i]) return true;
      if (latestParts[i] < currentParts[i]) return false;
    }
    return latestParts.length > currentParts.length;
  }
}

class StoreVersion {
  final String version;
  final bool isMandatory;
  final String storeUrl;
  
  StoreVersion({
    required this.version,
    required this.isMandatory,
    required this.storeUrl,
  });
}
```

**Explanation:**

- **Store Version Checking**: Detects when new version is available in stores, prompting users to update.
- **Mandatory Updates**: Critical for breaking API changes. Force users to update before app usage.
- **App Store Lookup**: Official iTunes API provides version info. Play Store has no official API; use backend or third-party services.
- **Version Comparison**: Semantic version comparison (1.2.3 vs 1.2.2).

---

## **43.5 Beta Testing and Distribution**

Managing internal and external testing tracks before production release.

### **TestFlight Configuration**

```ruby
# ios/fastlane/Fastfile additions
desc "Distribute to internal testers"
lane :internal do
  build
  
  upload_to_testflight(
    skip_waiting_for_build_processing: false,
    wait_processing_timeout_duration: 1800, # 30 minutes
    distribute_only: false,
    groups: ["Internal Team"],
    changelog: changelog_from_git_commits(
      commits_count: 10,
      pretty: "- %s"
    )
  )
end

desc "Distribute to external beta"
lane :external_beta do
  # Ensure app is ready for review
  precheck
  
  upload_to_testflight(
    beta_app_review_info: {
      contact_email: "beta@example.com",
      contact_first_name: "John",
      contact_last_name: "Doe",
      contact_phone: "+1 555 123 4567",
      demo_account_name: "demo@example.com",
      demo_account_password: "password123"
    },
    distribute_external: true,
    groups: ["External Beta"],
    submit_beta_review: true
  )
end
```

**Explanation:**

- **Internal Testers**: Up to 100 team members, no review required.
- **External Testers**: Up to 10,000 users, requires beta app review (lighter than production review).
- **Demo Account**: Required if app has login. Apple testers need credentials to review.
- **Changelog**: Auto-generated from git commits for release notes.

### **Google Play Testing Tracks**

```ruby
# android/fastlane/Fastfile additions
desc "Deploy to internal testing"
lane :internal do
  gradle(task: "bundleRelease")
  
  upload_to_play_store(
    track: "internal",
    release_status: "completed",
    skip_upload_metadata: true,
    skip_upload_images: true,
    skip_upload_screenshots: true,
    aab: "build/app/outputs/bundle/release/app-release.aab"
  )
end

desc "Deploy to closed beta"
lane :beta do
  gradle(task: "bundleRelease")
  
  upload_to_play_store(
    track: "beta",
    release_status: "completed",
    aab: "build/app/outputs/bundle/release/app-release.aab"
  )
end

desc "Promote beta to production"
lane :promote do
  upload_to_play_store(
    track: "beta",
    track_promote_to: "production",
    skip_upload_changelogs: false,
    skip_upload_metadata: true,
    skip_upload_images: true,
    skip_upload_screenshots: true
  )
end
```

**Explanation:**

- **Internal Testing**: Immediate availability to up to 100 testers.
- **Closed Beta**: Invite-only testing for larger groups.
- **Open Beta**: Public beta listing on Play Store.
- **Promotion**: Move tested beta builds to production without rebuilding.

---

## **Chapter Summary**

In this chapter, we covered the complete build and release pipeline for Flutter applications:

### **Key Takeaways:**

1. **Android App Bundle (AAB)**: Modern distribution format enabling dynamic delivery. Configure `android.bundle` blocks for language/density/ABI splits. Sign with release keystore using environment variables.

2. **iOS Code Signing**: Use Fastlane Match to share certificates and provisioning profiles securely via encrypted git. Automate keychain management in CI/CD.

3. **Fastlane Automation**: Standard tool for mobile DevOps. Handles building, signing, and uploading to both stores. Configure lanes for different environments (internal, beta, production).

4. **CI/CD Pipelines**: GitHub Actions for flexibility, Codemagic for Flutter-optimized workflows. Both require macOS runners for iOS builds. Use secrets management for credentials.

5. **OTA Updates**: Shorebird replaces CodePush for Dart hotfixes. Use in-app update checks for mandatory store updates. Understand limitations (cannot update native code).

6. **Beta Testing**: TestFlight for iOS (internal 100 users, external 10,000). Google Play Internal/Closed/Open tracks for Android staged rollouts.

7. **Version Management**: Automate version codes using build numbers or timestamps. Keep semantic versioning (versionName) meaningful for users.

### **Release Checklist:**
- ✅ Version bumped in pubspec.yaml (versionName + versionCode/buildNumber)
- ✅ Changelog updated with user-facing changes
- ✅ Android AAB signed with release keystore
- ✅ iOS certificates valid and not expiring soon
- ✅ Integration tests passing in CI
- ✅ Security scan completed (dependencies, secrets)
- ✅ Beta testers validated on both platforms
- ✅ App store metadata (screenshots, descriptions) updated
- ✅ Privacy policy URL accessible
- ✅ OTA patches signed and tested (if using Shorebird)

---

## **Next Steps**

In the next chapter, **Chapter 44: Project 1 - E-Commerce App**, we will begin the first comprehensive real-world project. You'll architect a full e-commerce application implementing Clean Architecture with BLoC pattern, featuring a product catalog with search and filters, shopping cart state management, payment gateway integration, and order tracking with push notifications.

---

**End of Chapter 43**